<a href="https://colab.research.google.com/github/hnsaavedraa/Computacion_Paralela/blob/master/practica3/Practica_3_CUDA_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title cuda Install
!echo CUDA AND NVIDIA INSTALLATION
!echo NOTE: THERE IS A QUESTION IN THE INSTALLATION, PLEASE DO NOT FORGET TO ANSWER IT (YOU CAN CHOOSE Y)
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb;
!dpkg -i cuda-repo-ubuntu1604_8.0.61-1_amd64.deb;
!apt-get update -qq;
!apt-get install cuda-8.0;
!ln -sf /usr/local/cuda-8.0 /usr/local/cuda

import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda/lib'

!apt-get install gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

CUDA AND NVIDIA INSTALLATION
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `echo NOTE: THERE IS A QUESTION IN THE INSTALLATION, PLEASE DO NOT FORGET TO ANSWER IT (YOU CAN CHOOSE Y)'
--2020-05-17 17:31:33--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2690 (2.6K) [application/x-deb]
Saving to: ‘cuda-repo-ubuntu1604_8.0.61-1_amd64.deb’

cuda-repo-ubuntu160 100%[===================>]   2.63K  --.-KB/s    in 0s      

2020-05-17 17:31:33 (158 MB/s) - ‘cuda-repo-ubuntu1604_8.0.61-1_amd64.deb’ saved [2690/2690]

Selecting previously unselected package cuda-repo-ubuntu1604.
(Reading database ... 144433 files and directories currently 

In [0]:
!wget https://raw.githubusercontent.com/hnsaavedraa/Computacion_Paralela/master/practica1/720.jpg
!wget https://raw.githubusercontent.com/hnsaavedraa/Computacion_Paralela/master/practica1/1080.jpg
!wget https://raw.githubusercontent.com/hnsaavedraa/Computacion_Paralela/master/practica1/4k.jpg
!wget https://raw.githubusercontent.com/hnsaavedraa/Computacion_Paralela/master/practica1/stb_image/stb_image.h
!wget https://raw.githubusercontent.com/hnsaavedraa/Computacion_Paralela/master/practica1/stb_image/stb_image_write.h
!wget https://raw.githubusercontent.com/hnsaavedraa/Computacion_Paralela/master/practica1/stb_image/stb_image_resize.h

!mkdir src
!mkdir src/stb_image
!mv *.jpg src/
!mv *.h src/stb_image/

--2020-05-17 17:33:15--  https://raw.githubusercontent.com/hnsaavedraa/Computacion_Paralela/master/practica1/720.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 465450 (455K) [image/jpeg]
Saving to: ‘720.jpg’

720.jpg             100%[===================>] 454.54K  --.-KB/s    in 0.02s   

2020-05-17 17:33:16 (18.1 MB/s) - ‘720.jpg’ saved [465450/465450]

--2020-05-17 17:33:18--  https://raw.githubusercontent.com/hnsaavedraa/Computacion_Paralela/master/practica1/1080.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 359455 (351K) [ima

In [0]:
%cd  /usr/local/cuda/samples/1_Utilities/deviceQuery/
!ls
!make
!./deviceQuery
!nvcc --version
%cat /usr/local/cuda/samples/1_Utilities/deviceQuery/deviceQuery.cpp

/usr/local/cuda-10.1/samples/1_Utilities/deviceQuery
deviceQuery.cpp  Makefile  NsightEclipse.xml  readme.txt
/usr/local/cuda-10.1/bin/nvcc -ccbin g++ -I../../common/inc  -m64    -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_75,code=compute_75 -o deviceQuery.o -c deviceQuery.cpp
/usr/local/cuda-10.1/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_75,code=compute_75 -o

In [0]:
%cd /

/


In [0]:
%%cuda --name blur_effect.cu

#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include <stdbool.h>
#include <math.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <helper_cuda.h>

#define STB_IMAGE_IMPLEMENTATION
#define STB_IMAGE_WRITE_IMPLEMENTATION

#include "stb_image/stb_image.h"
#include "stb_image/stb_image_write.h"

__device__ int myMax(int num1, int num2)
{
	return (num1 > num2) ? num1 : num2;
}

__device__ int myMin(int num1, int num2)
{
	return (num1 > num2) ? num2 : num1;
}

//Metodo que calcula el radio de las cajas (br) para el algoritmo boxblur
void boxesForGauss(int sigma, int n, double *sizes) 
{

	double wIdeal = sqrt((12 * sigma * sigma / n) + 1); 
	double wl = floor(wIdeal);
	if (fmod(wl, 2.0) == 0.0)
	{
		wl--;
	}
	double wu = wl + 2;

	double mIdeal = (12 * sigma * sigma - n * wl * wl - 4 * n * wl - 3 * n) / (-4 * wl - 4);
	double m = round(mIdeal);
	for (int i = 0; i < n; i++)
	{
		*(sizes + i) = (i < m ? wl : wu);
	}
}

//Metodo ejecutado por los hilos
__global__ void parallelProcess(int *scl, int *tcl, int w, int h, int r, int totalThreads)
{
	int index = (blockDim.x * blockIdx.x) + threadIdx.x;
	int init_i = ((w*h)/totalThreads) * index;
	int end_i;
	if(index == totalThreads-1){
			end_i = ((w*h)- init_i);
	}
	else{
			end_i = ((w*h)/(totalThreads));
	}
	int i = init_i / w;
	int j = init_i % w;
	int control = 0;

	while(control < end_i){
			int val = 0;
			for (int ix = j - r; ix < j + r + 1; ix++)
			{
				int x = myMin(w - 1, myMax(0, ix));
				val += *(tcl + (i * w + x));
			}

			*(scl + (i * w + j)) = val / (r + r + 1);
			j++;
			if(j == w){
					j = 0;
					i++;
			}
			if(i == h){
					break;
			}
			control++;
	}

	int a = init_i / w;
	int b = init_i % w;
	int control_2 = 0;

	while(control_2 < end_i){
			int val = 0;
			for (int iy = a - r; iy < a + r + 1; iy++)
			{
				int y = myMin(h - 1, myMax(0, iy));
				val += *(scl + (y * w + b));
			}

			*(tcl + (a * w + b)) = val / (r + r + 1);
			b++;
			if(b == w){
					b = 0;
					a++;
			}
			if(a == h){
					break;
			}
			control_2++;
	}
}

//Metodo donde se crean los hilos y se asigna el trabajo de cada uno
void boxBlur(int *scl, int *tcl, int w, int h, int r, int blocksPerGrid, int threadsPerBlock)
{
	for (int i = 0; i < (w * h); i++)
	{
		int aux = *(scl + i);
		*(tcl + i) = aux;
	}

	int *d_scl, *d_tcl;
	cudaError_t err = cudaSuccess;

	err = cudaMalloc((void **)&d_scl, sizeof(int)*w*h);
	if (err != cudaSuccess){
			fprintf(stderr, "Failed to allocate device vector C (error code %s)!\n", cudaGetErrorString(err));
			exit(EXIT_FAILURE);
	}

	err = cudaMalloc((void **)&d_tcl, sizeof(int)*w*h);
	if (err != cudaSuccess){
			fprintf(stderr, "Failed to allocate device vector C (error code %s)!\n", cudaGetErrorString(err));
			exit(EXIT_FAILURE);
	}

	err = cudaMemcpy(d_scl, scl, sizeof(int)*w*h, cudaMemcpyHostToDevice);
	if (err != cudaSuccess){
			fprintf(stderr, "Failed to copy vector C from device to host (error code %s)!\n", cudaGetErrorString(err));
			exit(EXIT_FAILURE);
	}

	err = cudaMemcpy(d_tcl, tcl, sizeof(int)*w*h, cudaMemcpyHostToDevice);
	if (err != cudaSuccess){
			fprintf(stderr, "Failed to copy vector C from device to host (error code %s)!\n", cudaGetErrorString(err));
			exit(EXIT_FAILURE);
	}

	int totalThreads;
	totalThreads = blocksPerGrid * threadsPerBlock;
	parallelProcess<<<blocksPerGrid, threadsPerBlock>>>(d_scl, d_tcl, w, h, r, totalThreads);

	err = cudaMemcpy(scl, d_scl, sizeof(int)*w*h, cudaMemcpyDeviceToHost);
	if (err != cudaSuccess){
			fprintf(stderr, "Failed to copy vector from device to host (error code %s)!\n", cudaGetErrorString(err));
			exit(EXIT_FAILURE);
	}
	
	err = cudaMemcpy(tcl, d_tcl, sizeof(int)*w*h, cudaMemcpyDeviceToHost);
	if (err != cudaSuccess){
			fprintf(stderr, "Failed to copy vector from device to host (error code %s)!\n", cudaGetErrorString(err));
			exit(EXIT_FAILURE);
	}

	err = cudaFree(d_scl);
	if (err != cudaSuccess){
			fprintf(stderr, "Failed to free device vector C (error code %s)!\n", cudaGetErrorString(err));
			exit(EXIT_FAILURE);
	}

	err = cudaFree(d_tcl);
	if (err != cudaSuccess){
			fprintf(stderr, "Failed to free device vector C (error code %s)!\n", cudaGetErrorString(err));
			exit(EXIT_FAILURE);
	}
}

//Metodo que aplica las iteracion del algoritmo boxblur para aproximar Gaussian blur
void gaussBlur_3(int *scl, int *tcl, int w, int h, int r, int blocksPerGrid, int threadsPerBlock)
{

	double *bxs = (double *)malloc(sizeof(double) * 3);
	boxesForGauss(r, 3, bxs);

	boxBlur(scl, tcl, w, h, (int)((*(bxs)-1) / 2), blocksPerGrid, threadsPerBlock);
	boxBlur(tcl, scl, w, h, (int)((*(bxs + 1) - 1) / 2), blocksPerGrid, threadsPerBlock);
	boxBlur(scl, tcl, w, h, (int)((*(bxs + 2) - 1) / 2), blocksPerGrid, threadsPerBlock);
}


int main(int argc, char **argv)
{
	// Cargamos la imagen y definimos variables 
	int width, height, channels;
	char img_name[128];
	strcpy(img_name, argv[1]);
	char new_img_name[128];
	char *eptr;
	int kernel = atoi(argv[3]);
	double multiplicator = strtod(argv[4],&eptr);
	strcpy(new_img_name, argv[2]);
	unsigned char *img = stbi_load(img_name, &width, &height, &channels, 0);
	cudaSetDevice(0);
	cudaDeviceProp deviceProp;
	cudaGetDeviceProperties(&deviceProp, 0);
	int blocksPerGrid, threadsPerBlock, totalThreads;
	blocksPerGrid = deviceProp.multiProcessorCount;	
	threadsPerBlock = multiplicator *_ConvertSMVer2Cores(deviceProp.major, deviceProp.minor);
	totalThreads = blocksPerGrid * threadsPerBlock;

	printf("CUDA kernel launch with %d blocks of %d threads Total: %i \n", blocksPerGrid, threadsPerBlock, totalThreads);

	if (img == NULL)
	{
		printf("Error in loading the image\n");
		exit(1);
	}
	printf("Loaded image with a width of %dpx, a height of %dpx and %d channels\n", width, height, channels);

	// Almacenamos los valores de cada canal
	int img_size = width * height * channels;
	int *r = (int *)malloc(sizeof(int) * (img_size / 3));
	int *g = (int *)malloc(sizeof(int) * (img_size / 3));
	int *b = (int *)malloc(sizeof(int) * (img_size / 3));
	int i = 0;
	for (unsigned char *p = img; p != img + img_size; p += channels, i++)
	{
		*(r + i) = (uint8_t)*p;
		*(g + i) = (uint8_t) * (p + 1);
		*(b + i) = (uint8_t) * (p + 2);
	}

	// Instanciamos los canales de la imagen de salida
	int *r_target = (int *)malloc(sizeof(int) * (img_size / 3));
	int *g_target = (int *)malloc(sizeof(int) * (img_size / 3));
	int *b_target = (int *)malloc(sizeof(int) * (img_size / 3));
	int j = 0;

	//Aplicamos el algoritmo para cada canal
	gaussBlur_3(r, r_target, width, height, kernel, blocksPerGrid, threadsPerBlock);
	gaussBlur_3(g, g_target, width, height, kernel, blocksPerGrid, threadsPerBlock);
	gaussBlur_3(b, b_target, width, height, kernel, blocksPerGrid, threadsPerBlock);

	// Se reconstruye la imagen a partir de los canales procesados
	for (int i = 0; i < img_size / 3; i++)
	{
		img[j] = *(r_target + i);
		img[j + 1] = *(g_target + i);
		img[j + 2] = *(b_target + i);
		j += 3;
	}

	//Se crea la nueva imagen y liberamos memoria 
	stbi_write_jpg(new_img_name, width, height, channels, img, 100);
	free(r_target);
	free(g_target);
	free(b_target);
	free(r);
	free(g);
	free(b);
}

'File written in /content/src/blur_effect.cu'

In [0]:
!nvcc -Xcompiler -Wall /content/src/blur_effect.cu -o /content/src/blur -I "usr/local/cuda/samples/common/inc" -lm 

/content/src/stb_image/stb_image.h(4096): warning: variable "old_limit" was set but never used

/content/src/stb_image/stb_image.h(4975): warning: variable "idata_limit_old" was set but never used



In [0]:
!time /content/src/blur /content/src/720.jpg /content/src/out.jpg 15 4

CUDA kernel launch with 40 blocks of 256 threads Total: 10240 
Loaded image with a width of 1280px, a height of 720px and 3 channels

real	0m0.553s
user	0m0.276s
sys	0m0.176s


In [0]:
%%shell

img_init=('/content/src/720.jpg' '/content/src/1080.jpg' '/content/src/4k.jpg')
img_fin=('output720.jpg' 'output1080.jpg' 'output4k.jpg')
touch times.txt
FILE=/times.txt

echo "" > times.txt


for i in 0 1 2
do
	for kernel in 3 5 7 9 11 13 15
	do
		for NumHilos in 1
		do
			for intentos in 1
			do

				mytime="$(time ( time /content/src/blur ${img_init[i]} ${img_fin[i]} $kernel $NumHilos ) 2>&1 1>/dev/null )"
				jl=""
			
				echo "Tiempo para imagen" ${img_init[i]}  " con un kernel de " $kernel " y con  "  $NumHilos " hilos , intento numero" $intentos>> times.txt
				echo $mytime >> times.txt
				echo $jl >> times.txt
				echo "Tiempo para imagen" ${img_init[i]}  " con un kernel de " $kernel " y con  "  $NumHilos " hilos, intento numero" $intentos
				echo $mytime

			done

			
		done 
	done
	   
done

Tiempo para imagen /content/src/720.jpg  con un kernel de  3  y con   1  hilos, intento numero 1
real 0m0.417s user 0m0.234s sys 0m0.168s
Tiempo para imagen /content/src/720.jpg  con un kernel de  5  y con   1  hilos, intento numero 1
real 0m0.383s user 0m0.231s sys 0m0.140s
Tiempo para imagen /content/src/720.jpg  con un kernel de  7  y con   1  hilos, intento numero 1
real 0m0.387s user 0m0.224s sys 0m0.143s
Tiempo para imagen /content/src/720.jpg  con un kernel de  9  y con   1  hilos, intento numero 1
real 0m0.383s user 0m0.211s sys 0m0.155s
Tiempo para imagen /content/src/720.jpg  con un kernel de  11  y con   1  hilos, intento numero 1
real 0m0.382s user 0m0.217s sys 0m0.151s
Tiempo para imagen /content/src/720.jpg  con un kernel de  13  y con   1  hilos, intento numero 1
real 0m0.409s user 0m0.225s sys 0m0.166s
Tiempo para imagen /content/src/720.jpg  con un kernel de  15  y con   1  hilos, intento numero 1
real 0m0.413s user 0m0.241s sys 0m0.154s
Tiempo para imagen /content/src